In [ ]:
import pandas as pd
import numpy as np
import math
COUNTER = 0
df = pd.read_csv("nandi.csv")
#print(df)
Points = df.to_numpy()
Points
min(Points[:,1])
# points = [(0+j,0+i,0) for i in range(-1,2) for j in range(0,3)]
# print(points)
# Points = np.array(points,dtype=float)
# print(Points)
dau = 0.1  # Minimum accuracy of point cloud
N = Points.shape[0] # Total number of points
m = 2 # Empirical Value need to be checked for our case
x_min = min(Points[:,0]) - dau
x_max = max(Points[:,0]) + dau
y_min = min(Points[:,1]) - dau
y_max = max(Points[:,1]) + dau
S = ((x_max - x_min)*(y_max - y_min))/N
e = (m*S)**0.5 # cell length
n_x = math.ceil((x_max - x_min)/e)  # number of cells in x
n_y = math.ceil((y_max - y_min)/e)  # number of cells in y
e
def get_grid_id(i):
    point = Points[i,:]
    return n_x * math.floor((point[1]-y_min)/e) + math.ceil((point[0] - x_min)/e)
num_grids = n_x * n_y;
grid_id2points = [[] for i in range(num_grids+1)]
for i in range (N):grid_id2points[get_grid_id(i)].append(i)
def is_valid(l):
    a,b = l
    return (a>=0 and a<=n_x-1 and b>=0 and b<=n_y-1)
def generation_points(i, n=1):
    a,b = (i-1)//n_x , (i-1)%n_x
    l = [(a+of1,b+of2) for of2 in range(-n,n+1) for of1 in range(-n,n+1)]
    l = list(filter(is_valid,l))
    l = [a*n_x+b+1 for a,b in l]
    return l
def snn(ctr_pt_id,gen_id = 1):
    snn_pt_ids, gen = [], []
    grid_id = get_grid_id(ctr_pt_id)

    #gen_id = 1;
    while(gen_id<3):
        gen = generation_points(grid_id, gen_id)
        r_2 = (gen_id * e)**2
        for grid_id in gen:
            if(grid_id>n_x*n_y - 1):
                #print(grid_id)
                continue
            point_ids = grid_id2points[grid_id]
            x = Points[point_ids]-Points[ctr_pt_id]
            sphere_cond = np.sum(np.square(x),1)
            a_bool = sphere_cond <= r_2
            b_bool = sphere_cond > 0
            bool_array = a_bool & b_bool
            for i in range (bool_array.size):
                if(bool_array[i]):
                    snn_pt_ids.append(point_ids[i])

        if(len(snn_pt_ids)):
            break
        else:
            gen_id+=1

    return snn_pt_ids
Graph = [[[],[]] for a in range(N)]
Triangle = []
edges_queue = []

def add_edge_to_graph(e):
    global COUNTER
    a,b,c = e
    Graph[a][0].append(b)
    Graph[b][1].append(a)
    COUNTER = COUNTER+1
    if COUNTER%1000==0:
        print(COUNTER)


def add_triangle(e):
    Triangle.append(e)


def min_distance(p_id,l_ids):
    i = np.argmin(np.sum(np.square(Points[l_ids]-Points[p_id]),1))
    return l_ids[i]

def min_distance_traingle(pa_id,pb_id,l_ids):
    AC = np.sqrt(np.sum(np.square(Points[l_ids]-Points[pa_id]),1))
    BC = np.sqrt(np.sum(np.square(Points[l_ids]-Points[pb_id]),1))
    i = np.argmin(AC+BC)
    return l_ids[i]

def form_seed_traingle(pa_id):
    snn_pa = snn(pa_id)
    pb_id = min_distance(pa_id,snn_pa)
    snn_pb = snn(pb_id)
    union_ids = set(snn_pa + snn_pb)
    union_ids.discard(pa_id)
    union_ids.discard(pb_id)
    pc_id = min_distance_traingle(pa_id,pb_id,list(union_ids))
    ba = Points[pb_id]-Points[pa_id]
    ca = Points[pc_id]-Points[pa_id]
    p_order = [pa_id,pb_id,pc_id]
    if np.cross(ba,ca)[-1]<0:p_order[1],p_order[2]=p_order[2],p_order[1]
    for i in range(3):
        e = (p_order[i],p_order[(i+1)%3],p_order[(i+2)%3])
        add_edge_to_graph(e)
        edges_queue.append(e)
    return p_order
delaunay_threshold = np.pi/12
dihedral_threshold = np.cos(np.pi/2)
def calculate_angle(x,y):
    return np.arccos(np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y)+1e-12))

def get_angles(a,b,c):
    a,b,c = Points[[a,b,c]]
    return calculate_angle(b-a,c-a),calculate_angle(a-b,c-b),calculate_angle(a-c,b-c)
def delaunay_dihedral_constraint(e,l_ids):
    a,b,c = e
    result = []
    for v in l_ids:
        a_angle,b_angle,v_angle = get_angles(a,b,v)
        #print("for a,b,c,v=",e,v)
        a_pt, b_pt, c_pt, v_pt = Points[[a,b,c,v]]
        cos_beta = np.cos(calculate_angle(np.cross(b_pt-a_pt,c_pt-a_pt),np.cross(v_pt-a_pt,b_pt-a_pt)))
        #print("angles of triangle:",(a_angle,b_angle,v_angle),cos_beta,np.pi/6)
        if a_angle<delaunay_threshold or b_angle<delaunay_threshold or v_angle<delaunay_threshold:continue
        if cos_beta < dihedral_threshold: continue
        result.append(v)  #Here the error has occured, it was c instead of v
    return result
def illegal_point_edge_constrain(e,l_ids):
    a,b,c = e
    result = []
#     print("a,b,c",e)
    for v in l_ids:
#         print("v",v)
        if v==a or v==b or v==c:continue
        if (a in Graph[v][0]) and (a in Graph[v][1]):continue
        if (b in Graph[v][0]) and (b in Graph[v][1]):continue
#         print("v after",v)
        inner = True
        if(len(Graph[v][0]) == 0):
           inner = False;
        for outer_vertex in Graph[v][0]:
            inner = inner and (outer_vertex in Graph[v][1])
            if not inner:break
#             print("v after",v)
        if inner:continue  #Here the error has occured, I commented this line, as I did not understand the purpose of the perivous for loop, I did this
        result.append(v)
    return result
def check_constraints(e,l):

    l = delaunay_dihedral_constraint(e,l)
#     print("L after delauney:",l)
    l = illegal_point_edge_constrain(e,l)
#     print("L after illegal:",l)
    return l
def adjacent_vertices(e):
    a,b,c = e
    l = Graph[a][0]+Graph[a][1]+Graph[b][0]+Graph[b][1]
    return list(set(l))
#get_grid_id(a)
#get_points_from_grid_id(grid_id_a)
#SNN(a,r)
def candidate_set(e):
    a,b,c = e
    #step 1
    l = adjacent_vertices(e)
    l = check_constraints(e,l)
    if len(l)!=0:return l

    #step 2
    grid_id_a = get_grid_id(a)
    l = grid_id2points[grid_id_a]
    grid_id_b = get_grid_id(b)
    if grid_id_a!=grid_id_b: l = l + grid_id2points[grid_id_b]
#     print("step 2 before:",l)
    l =  check_constraints(e,l)
#     print("step 2:",l)
    if len(l)!=0:return l

    #step 3
    radius = 1
    for r in [radius,2*radius]:
        snn_a,snn_b = snn(a,r),snn(b,r)
        l = list(set(snn_a+snn_b))
        l = check_constraints(e,l)
        #print("step 3:",l)
        if len(l)!=0:return l

    #print("step final:",l)
    return l
lambdas = [0.6,0.2,0.2]
def find_optimal_vertex(e):
    a,b,c = e
    result = -1
    l = candidate_set(e)
#     print("cand_set:",l)
    max_score = -np.inf
    for v in l:
        cos_theta = np.cos(np.min(get_angles(a,b,v)))
        a_pt, b_pt, c_pt, v_pt = Points[[a,b,c,v]]
        cos_beta = np.cos(calculate_angle(np.cross(b_pt-a_pt,c_pt-a_pt),np.cross(v_pt-a_pt,b_pt-a_pt)))
        dist_term = np.linalg.norm(b-a)/(np.linalg.norm(a-v)+np.linalg.norm(b-v))
        score = lambdas[0]*cos_beta+lambdas[1]*0.5*(1/cos_theta)+lambdas[2]*dist_term
        if max_score<score:    #Here the error has occured, preivously it was max_score>score
            max_score = score
            result = v
    return result
def main(i):
    global Graph,edges_queue,Triangle
    #Graph = [[[],[]] for a in range(N)]
    #Triangle = []
    edges_queue = []
    add_triangle(form_seed_traingle(i))
    while len(edges_queue)!=0:
        e = edges_queue.pop(0)
        a,b,c = e
        if a in Graph[b][0]:continue
        v = find_optimal_vertex(e)
        #break
        if v==-1:continue

        add_edge_to_graph((b,a,v))
        add_edge_to_graph((a,v,b))
        add_edge_to_graph((v,b,a))
        add_triangle((b,a,v))
        edges_queue.append((a,v,b))
        edges_queue.append((v,b,a))
min_id = np.argmin(Points[:,0])
min_id = 7000
main(min_id)
for i in range(Points.shape[0]):
    num = get_grid_id(i)
    if num > n_x*n_y:
        print(f"id = {i}, Point = {Points[i]}")

#!pip install mpl_toolkits.mplot3d
#!pip install matplotlib
#!pip install ipympl

df_triangles = pd.DataFrame(Triangle, columns=['Vertex1', 'Vertex2', 'Vertex3'])

# Save the DataFrame to a CSV file
df_triangles.to_csv('triangles.csv', index=False)
